In [2]:
import os
import pandas as pd

# Create an empty dictionary to store the counts of each value
value_counts = {}

# Define the folder path
folder_path = 'c:/Users/pmcintyr/Desktop/MA3/SHS/pmcintyr.github.io/recensements'

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)
        # Read the Excel file
        df = pd.read_excel(file_path)
        # Get the second column
        column_values = df.iloc[:, 1]
        # Count the occurrences of each value in the second column
        for value in column_values:
            if value in value_counts:
                value_counts[value] += 1
            else:
                value_counts[value] = 1

# Convert the dictionary to a pandas DataFrame
result_df = pd.DataFrame(list(value_counts.items()), columns=['Value', 'Count'])

# Sort the DataFrame by the 'Count' column in descending order
result_df = result_df.sort_values('Count', ascending=False)

#print(result_df[0:100])

# Save the DataFrame to a CSV file
result_df.to_csv("task1.csv", index=False)

In [3]:
# Values to filter out
values_to_filter = ["pre","martherey","st jean","laurent","de bourg","francois","annee de naissance","pierre","rue du pont","chauerau","chailly","marteray","fre","halle","jn francois","derriere","id id","laurens","grand s jean","grand jean","devant","cheneau","cile de vant","st laurens","et","cile derriere","du marche","patud","francoise","gd st jean","hallede st laurent","haurent","grand fs jean","st laur","d halle","walle","jn jean","grand","paud","frand chene","cite derrier","moulins de pepinet","rue de martheray","cheneau bourg","marthony","marthorey","jennes","grand f jean","salud","de francois","du sont","cuchy","sejan","place t laurent","le pont","duz re","montee de st laurent","marthe","chemin de bourg","place du pont","marthery","f pierre","martherey e","dre","de pierre","la barre","grand fr jean","d etienne","valud","st fran","chavanne","ft pierre","grand sn jean","aisse","cite devriere","id .","luite dri","no de leur recepisse","cile derrier","flaurent","della barre","mererie","halle de laurent","du re","chaucrau .","l walle","marthere","theneau de bourg","e aisse","rue de francois","cete derriere","no des","pennes","marthoray","etienne","marberay","st laurant","cite derric","ctraz","chaucraie","bourge","ste pierre","theneau bourg","halle de s laurt","slaurent","petit gjean","montee de st monte","lite de vans","l hopital","rue du pre","de mercerie","pont","monorier","nerie","no es","monte st laurent","uve","grangeneuve","sdu","dean","ler","luchy","de st laurent","no de","grand sjean","tre","cite dessons","monbe clauron"]

# Filtering out specific values
result_df_filtered = result_df[~result_df['Value'].isin(values_to_filter)]

#take top 100
sort = result_df_filtered.sort_values('Count', ascending=False)[0:100]

# Save the DataFrame to a CSV file
sort.to_csv("task1.csv", index=False)